### 1. Titre du projet
Evaluation  des zones inondables dans le bassin du fleuve Gambie à partir d'un scénario de crue localisée - Cas de la station de Kédougou (Juillet 2024)

### 2. Contexte et objectif
Ce projet vise à localiser les zones critiques en aval de la station de Kédougou en croisant le MNT, l'occupation du sol et les données GHSL

### 3. Données utilisées
- MNT du bassin versant du fleuve Gambie
- Landcover ESA WorldCover 
- Données GHSL (population et surface bâtie)
- Données de crue ()
  * Nivau d'alerte (Juillet 2024):
  * Niveau observé :
  * Ecart modélisé

### 4. Technologies utilisées
- PCRaser: moteur principal de simulation pour la modélisation hydrologique, nottamment les fonctions accucapacityflux, accucapacitystate, spreadzone, ldd, etc.
- GDAL: convertion et manipulation de fichier raster (GeoTiff vers .map, reprojection, extraction de métadonnées).
- Numpy: manipulation efficace de matrice (ex. pour les conversions intermédiares ou calculs personnalisés).
- Matplotlib: visualisation des résultats(cartes des zones inondables, évolution temporelle).
- Jupyter notebook: environnement interactif pour tester, visualiser et documenter les simulations

### 5. Choix d'un modèle dynamique
Pour simuler le processus d'inondation sur le bassin Gambie, nous avons choisi d'utiliser un **modèle dynamique PCRaster** basé sur la classe `DynamicModel`.
#### Pourquoi un modèle dynamique?
- **Evolution temporelle** : Les phénomènes hydrologiques(écoulement, inondation, ruissellement) sont dépendants du temps. Le modèle dynamique permet d'intégrer cette évolution pas à pas.
- **Simulation réaliste** : Il permet d'observer l'impact cumulé des précipitation, de la topographie, de l'occupation du sol, etc sur la propagation de l'eau.
- **Flexibilité** : Grâce aux itérations temporelles, il est possible d'intégrer des paramètres variables dans le temps(ex. précipitations journalières, gestion des barrages, etc.).
Ce modèle est donc adapté pour reproduire et analyser le comportement spatial et  temporel des inondations dans le bassin.

### 6. Séparation des traitement statiques et dynamiques
Dans un souci de performation et de modularité, certains traitements geospataux lourds ou invariant ont été externalisés de la classe `DynamicModel`. L'objectif est de les exécuter une seule fois en amont, puis sauvegarder les résultats pour les réutilisés dans la simulation dynamiques

### 7. Traitements préalables effectués hors de `DynamicModel`

#### a. Importation des bibliothèques python

In [3]:
import pcraster as pcr
import numpy as np
import glob
import os
#import matplotlib.pyplot as plt
from osgeo import gdal, osr, gdalconst
#from tqdm import tqdm # pour une barre de progression
#from pyproj import Transformer
gdal.UseExceptions()

In [8]:
dem = gdal.Open('Data/basin_dem.tif') # le Modèle Numérique Terrestre
landcover = gdal.Open('Data/basincover.tif') # les données sur l'occupation du sol
ghsl_built = gdal.Open('Data/basin_built.tif') # les données GHSL de la distribution des surfaces urbanisées (bâties), exprimées en mètres carrés 2020
ghsl_pop = gdal.Open('Data/basin_pop.tif') # les données GHSL de la distribution spatiale de la population en 2020

#### b. Imporation des données et affichage des proprietés de nos dataset

In [9]:
# Fonction pour l'affichage des proprietés des datasets
def gdal_properties(input_path): # Affichier les proprietés pour chaque fichier
    if not os.path.exists(input_path):
        raise ValueError(f"Fichier introuvable")
    raster = gdal.Open(input_path)
    path = raster.GetDescription()
    width = raster.RasterXSize
    height = raster.RasterYSize
    projection = raster.GetProjection()
    geotransform = raster.GetGeoTransform()
    metadata = raster.GetMetadata()
    numband = raster.RasterCount
    driver = raster.GetDriver().ShortName
    print("#############################################")
    print('Propriétés')
    print(f"{path} a {width} x {height} comme dimension")
    print(f"Driver: {driver}")
    print(f"Nombre de bande: {numband}")
    print(f"Les metadonnées: {metadata}")

    srs = osr.SpatialReference() 
    srs.ImportFromWkt(projection)
    epsg_code = srs.GetAuthorityCode(None)
    unit_name = "Degree" if epsg_code == "4326" else srs.GetLinearUnitsName()
    unit_value = srs.GetLinearUnits()
    print(f"Unité de la projection: {unit_name} {unit_value}")

    if geotransform:
        
        print(f"Origine: ({geotransform[0]}, {geotransform[3]})")
        print(f"Size: ({geotransform[1]} {unit_name}, {geotransform[5]} {unit_name})")
    layerBand = raster.GetRasterBand(1)
    min_value, max_value = layerBand.ComputeRasterMinMax()
    print(f"Valeur minimale {min_value}")
    print(f"Valeur maximale {max_value}")
    print("#############################################")
    raster = None

In [10]:
basin_raster_paths = glob.glob('Data/basin*.tif') # Charger les fichiers geotiff
for path in basin_raster_paths:
    
    gdal_properties(path)
    print('\n')

#############################################
Propriétés
Data/basin_built.tif a 20060 x 17852 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: m 1.0
Origine: (-1721000.0, 2052780.0)
Size: (30.0 m, -30.0 m)
Valeur minimale 0.0
Valeur maximale 6599.0
#############################################


#############################################
Propriétés
Data/basin_dem.tif a 18137 x 12359 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: Degree 1.0
Origine: (-16.15022041715082, 14.68830289986267)
Size: (0.0002694945852358564 Degree, -0.0002694945852358564 Degree)
Valeur minimale -29.0
Valeur maximale 1530.0
#############################################


#############################################
Propriétés
Data/basin_pop.tif a 15897 x 13590 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection:

#### c. Alignement des rasters
Pour une cohérence spatiales, une interopérabilité entre les données et une optimisation des calculs, on va aligner nos rasters sur un raster de référence (même projection, geotransformation et résolution)

In [18]:
#fonction pour la reprojection
def align_rasters(reference_path, input_rasters, output_dir):
    """
    Aligner les rasters sur la référence GHSL built_surface
    """
    #configurer gdal pour optimiser les performances
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    gdal.SetConfigOption('GDAL_CACHEMAX', '512')

    # Vérifier/Convertir la référence en EPSG:32628
    ref_ds = gdal.Open(reference_path)
    ref_proj = ref_ds.GetProjection()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(ref_proj)

    if srs.GetAuthorityCode(None) != '32628':
        print('Conversion de la référence en EPSG:32628')
        temp_ref_path = os.path.join(output_dir, 'temp_ref_32628.tif')
        gdal.Warp(temp_ref_path, ref_ds, dstSRS="EPSG:32628")
        ref_df = None
        ref_ds = gdal.Open(temp_ref_path)
    

    # Paramètres de référence
    ref_proj = ref_ds.GetProjection()
    ref_gt = ref_ds.GetGeoTransform()
    x_size = ref_ds.RasterXSize
    y_size = ref_ds.RasterYSize

    # Calcul des bornes
    min_x = ref_gt[0]
    max_y = ref_gt[3]
    max_x = min_x + ref_gt[1] * x_size
    min_y = max_y + ref_gt[5] * y_size
    new_res = min(ref_gt[1], ref_gt[5])

    # Dossier de sortie
    os.makedirs(output_dir, exist_ok=True)

    # Méthodes de rééchantillonnage spécifique par type de données
    resample_methods = {
        'landcover': gdal.GRA_NearestNeighbour,
        'dem': gdal.GRA_Bilinear,
        'ghsl_pop': gdal.GRA_Cubic,
        'ghsl_built': gdal.GRA_Bilinear
    }

    # Traitement des rasters
    output_files = {}
    for name, input_path in tqdm(input_rasters.items()):
        if not os.path.exists(input_path):
            print(f"Fichier introuvable")
            continue
        output_path = os.path.join(output_dir, f'aligned_{name}.tif')

        # Options spécifiques por le warp
        warp_options = gdal.WarpOptions(
            format='GTiff',
            outputBounds = (min_x, min_y, max_x, max_y),
            xRes=abs(new_res),
            yRes=abs(new_res),
            dstSRS="EPSG:32628",
            resampleAlg=resample_methods.get(name, gdal.GRA_Bilinear),
            targetAlignedPixels=True,
            creationOptions=[
                'COMPRESS=DEFLATE',
                'PREDICTOR=2' if name in ['dem', 'ghsl_pop'] else 'PREDICTOR=1',
                'NUM_THREADS=ALL_CPUS'
                ],
                warpMemoryLimit=1021*1024*1024 # 1GB
            
        )
        print(f'Traitement en cours: {name}...')
        gdal.Warp(output_path, input_path, options=warp_options)
        output_files[name] = output_path
    # Nettoyage
    if 'temp_ref_path' in locals():
        os.remove(temp_ref_path)
    ref_ds = None
    
    return output_files

In [19]:
data = {
    'reference': 'Data/basin_built.tif',
    'landcover': 'Data/basincover.tif',
    'dem': 'Data/basin_dem.tif',
    'ghsl_pop': 'Data/basin_pop.tif',
    'ghsl_built': 'Data/basin_built.tif'
}
# Exéution de l'alignement
results = align_rasters(
    reference_path=data['reference'],
    input_rasters={k: data[k] for k in ['landcover', 'dem', 'ghsl_pop', 'ghsl_built']},
    output_dir='Data'
)

print("\nRésultats de l'alignement:")
for name, path in results.items():
    print(f"{name.upper():<15} {path}")

Conversion de la référence en EPSG:32628


  0%|                                                     | 0/4 [00:00<?, ?it/s]

Triatement en cours: landcover...


 25%|███████████▎                                 | 1/4 [00:19<00:59, 19.77s/it]

Triatement en cours: dem...


...

Triatement en cours: ghsl_pop...


 75%|█████████████████████████████████           | 3/4 [04:32<01:52, 112.47s/it]

Triatement en cours: ghsl_built...


100%|█████████████████████████████████████████████| 4/4 [05:45<00:00, 86.37s/it]



Résultats de l'alignement:
LANDCOVER       Data/aligned_landcover.tif
DEM             Data/aligned_dem.tif
GHSL_POP        Data/aligned_ghsl_pop.tif
GHSL_BUILT      Data/aligned_ghsl_built.tif


#### d. Verification de l'alignement des rasters

In [13]:
aligned_raster_paths = glob.glob('Data/aligned*.tif')
for path in aligned_raster_paths:
    
    gdal_properties(path)
    print('\n')

#############################################
Propriétés
Data/aligned_ghsl_built.tif a 21880 x 15706 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: metre 1.0
Origine: (219033.12946546363, 1848953.0947196095)
Size: (31.099407846864068 metre, -31.099407846864068 metre)
Valeur minimale 0.0
Valeur maximale 6599.0
#############################################


#############################################
Propriétés
Data/aligned_ghsl_pop.tif a 21880 x 15706 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: metre 1.0
Origine: (219033.12946546363, 1848953.0947196095)
Size: (31.099407846864068 metre, -31.099407846864068 metre)
Valeur minimale -20.035439608458702
Valeur maximale 439.70852641010066
#############################################


#############################################
Propriétés
Data/aligned_landcover.tif a 21880 x 15706 comme dimension
Dr

#### e. Conversion
les coordonées GPS(longitude, latitude) des stations Hydrometrique doivent être convertis en coordonnées image(ligne, colonne).
La conversion est nécessaire lorsqu'on travaille avec des rasters(comme un MNT, un LDD).
- Station Kégoudou (-12.1833, 12.55)
- Station Mako (-12.35, 12.8667)

In [35]:
#définition d'une fonction de conversion des coordonnées géographiques en indice de pixel ou inversement
def geoToPixel(x, y, geotransform, inverse=False):
    # D'abord on va convertir les coordonnées géographiques en coordonnées projetées
    def gps_to_utm(x, y): #fonction pour convertir les coordonnées géographiques en coordonnées projetées
        transformer_wgs = Transformer.from_crs("EPSG:4326", "EPSG:32628", always_xy=True)
        
        x_proj, y_proj = transformer_wgs.transform(x, y)
        return [x_proj, y_proj]
        
    if inverse is True:
        print("Convertir Pixel à Géo")
        xcoord = geotransform[0] + x * geotransform[1]
        ycoord = geotransform[3] + y * geotransform[5]
        
    elif inverse is False:
       x_utm, y_utm = gps_to_utm(x, y)
       print("Convertir Géo a Pixel")
       xcoord = int((x_utm - geotransform[0]) / geotransform[1])
       ycoord = int((y_utm - geotransform[3]) / geotransform[5])
    else:
        print("le raster n'a pas de trasnformation")
    return xcoord, ycoord
#tester la fonction sur notre raster
# Choisir un GeoTransformation à partir de nos raster
geotransform = gdal.Open('Data/aligned_dem.tif').GetGeoTransform()
kedougou = geoToPixel(-12.35, 12.8667, geotransform)
mako = geoToPixel(-12.35, 12.8667, geotransform)
print(f"Station Kédougou en Coordonées image {kedougou}")
print(f"Station Mako en Coordonées image {mako}")

Convertir Géo a Pixel


(18282, 13668)

#### f. Traduction les fichiers GeoTiff déja alignés en fichier pcraster(.map)

In [28]:
# Définition de la fonction de conversion
def convert_to_pcraster(input_filename, output_filename, gdal_type,value_scale):

    gdal.SetConfigOption("PCRASTER_VALUESCLAE", value_scale)
    gdal.UseExceptions()
    src_ds = gdal.Open(input_filename, gdalconst.GA_ReadOnly)
    
    gdal.Translate(output_filename, src_ds, format="PCRaster", outputType=gdal_type, metadataOptions=[f"VALUESCALE={value_scale}"])#creationOptions=[f"VALUESCALE={value_scale}"])
    print(f"Converti : {input_filename} en {output_filename}")
    
    #GDAL Translate
    #dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None
    


In [29]:
# Exécution de de la fonction de conversion des fichiers .tif en .map
input_filenames = {
    'Data/aligned_dem.tif': {
        'output': 'Data/dem.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    },
    'Data/aligned_landcover.tif': {
        'output': 'Data/landcover.map',
        'value_scale': 'VS_NOMINAL',
        'gdal_type': gdalconst.GDT_Byte
    },
'Data/aligned_ghsl_built.tif': {
        'output': 'Data/ghsl_built.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    },
'Data/aligned_ghsl_pop.tif': {
        'output': 'Data/ghsl_pop.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    }
}
# Lancer les conversations
for filename, infos in input_filenames.items():
    convert_to_pcraster(input_filename=filename,
                       output_filename=infos['output'],
                       value_scale=infos['value_scale'],
                       gdal_type=infos['gdal_type'])


Converti : Data/aligned_dem.tif en Data/dem.map
Converti : Data/aligned_landcover.tif en Data/landcover.map
Converti : Data/aligned_ghsl_built.tif en Data/ghsl_built.map
Converti : Data/aligned_ghsl_pop.tif en Data/ghsl_pop.map


#### g. Génération de LDD (Local Drain Direction)
Le LDD est une composante fondamentale dans toute modélisation hydrologique. Il représente, pour chaque cellule du MNT, la direction dans laquelle l'eau s'écoule vers une cellule voisine. Cette structure est indispensable pour simuler le ruissellement, l'accumulation ou les inondations.

In [9]:
#pcr.setclone('Data/dem.map')
#os.environ['PCRASTER_MEMORY_LIMIT'] = "6GB"

dem = pcr.readmap('Data/ghsl_built.map')
#ldd = pcr.lddcreate(dem, 1e13, 1e13, 1e13, 1e13)
#pcr.report(ldd, 'Data/ldd.map')

MemoryError: std::bad_alloc

In [6]:
dem = gdal.Open('Data/aligned_dem.tif')
band = dem.GetBand(1)

AttributeError: 'Dataset' object has no attribute 'GetBand'

In [5]:
dir(dem)

['AbortSQL',
 'AddBand',
 'AddFieldDomain',
 'AddRelationship',
 'AdviseRead',
 'BeginAsyncReader',
 'BuildOverviews',
 'ClearStatistics',
 'Close',
 'CommitTransaction',
 'CopyLayer',
 'CreateLayer',
 'CreateLayerFromGeomFieldDefn',
 'CreateMaskBand',
 'DeleteFieldDomain',
 'DeleteLayer',
 'DeleteRelationship',
 'Destroy',
 'EndAsyncReader',
 'ExecuteSQL',
 'FlushCache',
 'GetDescription',
 'GetDriver',
 'GetFieldDomain',
 'GetFieldDomainNames',
 'GetFileList',
 'GetGCPCount',
 'GetGCPProjection',
 'GetGCPSpatialRef',
 'GetGCPs',
 'GetGeoTransform',
 'GetLayer',
 'GetLayerByIndex',
 'GetLayerByName',
 'GetLayerCount',
 'GetMetadata',
 'GetMetadataDomainList',
 'GetMetadataItem',
 'GetMetadata_Dict',
 'GetMetadata_List',
 'GetName',
 'GetNextFeature',
 'GetProjection',
 'GetProjectionRef',
 'GetRasterBand',
 'GetRefCount',
 'GetRelationship',
 'GetRelationshipNames',
 'GetRootGroup',
 'GetSpatialRef',
 'GetStyleTable',
 'GetSubDatasets',
 'GetSummaryRefCount',
 'GetThreadSafeDataset',
